In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Plot
plot_directory = 'result/plot_set'

In [3]:
# colors = [
#     '#e6194B',
#     '#f58231',
#     '#9A6324',
#     '#911eb4',
#     '#3cb44b',
#     '#f032e6',
#     '#4363d8',
# ]

In [4]:
colors = [
	'#FF0000',
	'#00FFFF',
	'#0000FF',
	'#00008B',
	'#ADD8E6',
	'#800080',
	'#7FFFD4',
	'#008000',
	'#FF00FF',
	'#FFC0CB',
	'#C0C0C0',
	'#FFA500',
	'#000000',
	'#800000',
]

In [5]:
def load_json_file(file_path):
	try:
		with open(file_path, 'r') as file:
			data = json.load(file)
		return data
	
	except Exception as e:
		print(f"An error occurred while loading the JSON file: {e}")
		return None

In [6]:
def extract_fpss(metric_list):
	return list(metric_list[list(metric_list.keys())[0]][0]['metric'].keys())

In [7]:
def to_accuracy_vector(accuracy_result_seq, fpss, type='F1'):
	accuracy_vector = []
	for fps in fpss:
		accuracy_vector.append(accuracy_result_seq[fps][type])
	
	return accuracy_vector

In [8]:
def plot_scatter(xs, ys, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	plt.scatter(xs, ys, c=colors[0], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [9]:
def plot_scatter_label(xs, ys, labels, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	for i in range(len(xs)):
		plt.scatter(xs[i], ys[i], c=colors[labels[i]], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [10]:
def filter_list_index(input_list, indices_to_remove):
	return [item for i, item in enumerate(input_list) if i not in indices_to_remove]

In [11]:
def round_float_to_sigfigs(number, sigfigs):
	return round(number, sigfigs)

In [12]:
def normalize_list(input_list):
    # Check if the list is not empty
    if not input_list:
        return []

    min_val = min(input_list)
    max_val = max(input_list)

    # Handle the case where all elements in the list are the same
    if min_val == max_val:
        return [0.5] * len(input_list)

    return [(x - min_val) / (max_val - min_val) for x in input_list]


## Plot

In [13]:
omv_features = ["Left-Top", "Right-Top", "Left-Bottom", "Right-Bottom", "Object-Amount", "Confidence", "IOU"]

In [14]:
plot_filenames = sorted(os.listdir(plot_directory))
plot_video_names = sorted(list(set([f.split('_')[0] for f in plot_filenames])))

In [15]:
fpss = extract_fpss(load_json_file(os.path.join(plot_directory, plot_video_names[0] + "_Accuracy_Result.json")))

In [16]:
omv_videos = []
acc_videos = []

for v in plot_video_names:
	omv_dict = {}
	for fps in fpss:
		omv_dict[fps] = []
	acc_list = []

	accuracy_result = load_json_file(os.path.join(plot_directory, v + "_Accuracy_Result.json"))
	movement_result = load_json_file(os.path.join(plot_directory, v + "_Movement_Result.json"))

	for class_idx in list(accuracy_result.keys()):
		for i in range(len(accuracy_result[class_idx])):
			accuracy_vector = to_accuracy_vector(accuracy_result[class_idx][i]['metric'], fpss)
			acc_list.append(accuracy_vector)

			for fps in fpss:
				movement_vector = movement_result[class_idx][i]['movement'][fps]
				omv_dict[fps].append(movement_vector)
	
	omv_videos.append(omv_dict)
	acc_videos.append(acc_list)

In [17]:
# for i in range(len(plot_video_names)):
# 	video_name = plot_video_names[i]
# 	for j in range(len(omv_videos[0][fpss[0]][0])):
# 		for k in range(len(fpss)):
# 			fps = fpss[k]
			
# 			omv_fps = list(np.array(omv_videos[i][fps])[:, j])
# 			acc_fps = list(np.array(acc_videos[i])[:, k])

# 			# Remove Outliers
# 			outlier_index = [l for l in range(len(omv_fps)) if omv_fps[l] == -1]
# 			omv_fps_clean = filter_list_index(omv_fps, outlier_index)
# 			acc_fps_clean = filter_list_index(acc_fps, outlier_index)

# 			title = f'{video_name}; OMV Feature {j} ({omv_features[j]}); FPS: {fps}'

# 			correlation_matrix = np.corrcoef(omv_fps_clean, acc_fps_clean)
# 			correlation_coefficient = correlation_matrix[0, 1]
# 			print(f"{title} -> Corr: {round_float_to_sigfigs(correlation_coefficient, 3)}")
# 			# plot_scatter(omv_fps_clean, acc_fps_clean, 'OMV Feature', 'ACC', title)

# 		print("")

In [18]:
def iterate_combinations_7vars(total_sum=10):
    combinations = []
    for x1 in range(total_sum + 1):
        for x2 in range(total_sum - x1 + 1):
            for x3 in range(total_sum - x1 - x2 + 1):
                for x4 in range(total_sum - x1 - x2 - x3 + 1):
                    for x5 in range(total_sum - x1 - x2 - x3 - x4 + 1):
                        for x6 in range(total_sum - x1 - x2 - x3 - x4 - x5 + 1):
                            x7 = total_sum - x1 - x2 - x3 - x4 - x5 - x6
                            combinations.append((x1, x2, x3, x4, x5, x6, x7))
    return combinations

In [19]:
combinations = iterate_combinations_7vars(10)

In [20]:
best_average = -1.
best_combination = None

video_results = []
for combination in combinations:
	for i in range(len(plot_video_names)):
		fps_results = []
		for k in range(len(fpss)):
			fps = fpss[k]
			
			omv_fps_f0 = list(np.array(omv_videos[i][fps])[:, 0])
			outlier_index = [l for l in range(len(omv_fps_f0)) if omv_fps_f0[l] == -1]
			omv_fps_f0_clean = filter_list_index(omv_fps_f0, outlier_index)

			omv_weight_sum = combination[0] * np.array(normalize_list(omv_fps_f0_clean))
			for j in range(1, len(combination)):
				omv_fps = list(np.array(omv_videos[i][fps])[:, j])
				omv_fps_clean = filter_list_index(omv_fps, outlier_index)

				omv_weight_sum += combination[j] * np.array(normalize_list(omv_fps_clean))
			
			acc_fps = list(np.array(acc_videos[i])[:, k])
			acc_fps_clean = filter_list_index(acc_fps, outlier_index)
			
		
			correlation_matrix = np.corrcoef(omv_weight_sum, acc_fps_clean)
			correlation_coefficient = correlation_matrix[0, 1]

			fps_results.append(correlation_coefficient)
		video_results.append(np.average(np.array(fps_results)))
	combination_average_corr = np.average(np.array(video_results))

	if combination_average_corr > best_average:
		best_average = combination_average_corr
		best_combination = combination

Processing Combination: (0, 0, 0, 0, 0, 0, 10)
0.2953515671838627
Processing Combination: (0, 0, 0, 0, 0, 1, 9)
0.30648467831272475
Processing Combination: (0, 0, 0, 0, 0, 2, 8)
0.3181983463097586
Processing Combination: (0, 0, 0, 0, 0, 3, 7)
0.3302602606018404
Processing Combination: (0, 0, 0, 0, 0, 4, 6)
0.34216918034847515
Processing Combination: (0, 0, 0, 0, 0, 5, 5)
0.35297356160496524
Processing Combination: (0, 0, 0, 0, 0, 6, 4)
0.3614355152005393
Processing Combination: (0, 0, 0, 0, 0, 7, 3)
0.3667475244780752
Processing Combination: (0, 0, 0, 0, 0, 8, 2)
0.36884747860258416
Processing Combination: (0, 0, 0, 0, 0, 9, 1)
0.36812819484217296
Processing Combination: (0, 0, 0, 0, 0, 10, 0)
0.3651311895134492
Processing Combination: (0, 0, 0, 0, 1, 0, 9)
0.36077703733324934
Processing Combination: (0, 0, 0, 0, 1, 1, 8)
0.35927270983349724
Processing Combination: (0, 0, 0, 0, 1, 2, 7)
0.3602147782046825
Processing Combination: (0, 0, 0, 0, 1, 3, 6)
0.363198095148128
Processing Combin